# RNA-seq calling and QC

This document shows the use of various modules to prepare reference data, perform RNA-seq calling, expression level quantification and quality control. In particular,

1. [`reference_data.ipynb`](../data_preprocessing/reference_data.html)
2. [`RNA_calling.ipynb`](calling/RNA_calling.html)
3. [`readCount_QC.ipynb`](QC/readCount_QC.html)

A minimal working example is available on [Google Drive](https://drive.google.com/drive/u/0/folders/11kQv7PXozsKkgeqADH-28bC_kZ-w_oHo).

## Download reference data (first time use)

In [ ]:
sos run reference_data download_hg_reference --cwd reference_data
sos run reference_data download_gene_annotation --cwd reference_data
sos run reference_data download_ercc_reference --cwd reference_data

## Reformat reference data (first time use)

In [ ]:
sos run reference_data.ipynb hg_reference \
    --cwd reference_data \
    --ercc-reference reference_data/ERCC92.fa \
    --hg-reference reference_data/GRCh38_full_analysis_set_plus_decoy_hla.fa \
    --container container/rna_quantification.sif

**Notice: you need to know if your RNA-seq data is stranded or unstranded. If it is stranded, you should add `--is-stranded` to the command below.**

In [ ]:
sos run reference_data.ipynb gene_annotation \
    --cwd reference_data \
    --ercc-gtf reference_data/ERCC92.gtf \
    --hg-gtf reference_data/Homo_sapiens.GRCh38.103.chr.gtf \
    --hg-reference reference_data/GRCh38_full_analysis_set_plus_decoy_hla.noALT_noHLA_noDecoy_ERCC.fasta \
    --container container/rna_quantification.sif

## Index reference data

In [ ]:
sos run reference_data.ipynb STAR_index \
    --cwd reference_data \
    --hg-reference reference_data/GRCh38_full_analysis_set_plus_decoy_hla.noALT_noHLA_noDecoy_ERCC.fasta \
    --hg-gtf reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.ERCC.gtf \
    --container container/rna_quantification.sif \
    --mem 40G

In [ ]:
sos run reference_data.ipynb RSEM_index \
    --cwd reference_data \
    --hg-reference reference_data/GRCh38_full_analysis_set_plus_decoy_hla.noALT_noHLA_noDecoy_ERCC.fasta \
    --hg-gtf reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.ERCC.gtf \
    --container container/rna_quantification.sif \
    --mem 40G

## Perform data quality summary via `fastqc`

In [ ]:
sos run RNA_calling.ipynb fastqc \
    --cwd output/rnaseq/fastqc \
    --samples data/sample_fastq.list \
    --data-dir data \
    --container container/rna_quantification.sif

## Call gene-level RNA expression via `rnaseqc`

In [ ]:
sos run RNA_calling.ipynb rnaseqc_call \
    --cwd output/rnaseq \
    --samples data/sample_fastq.list \
    --data-dir data \
    --fasta_with_adapters_etc TruSeq3-PE.fa \
    --STAR-index reference_data/STAR_Index/ \
    --gtf reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.gene.ERCC.gtf \
    --container container/rna_quantification.sif \
    --mem 40G

## Call transcript level RNA expression via `RSEM`

In [ ]:
sos run RNA_calling.ipynb rsem_call \
    --cwd output/rnaseq \
    --samples data/sample_fastq.list \
    --data-dir data \
    --fasta_with_adapters_etc TruSeq3-PE.fa \
    --STAR-index reference_data/STAR_Index/ \
    --RSEM-index reference_data/RSEM_Index/ \
    --container container/rna_quantification.sif \
    --mem 40G